# LoRA

LoRA (Low-Rank Adaptation) is a technique for fine-tuning large pre-trained models by learning low-rank updates. It allows adapting a model efficiently without requiring full fine-tuning, saving computation and storage.

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')
write_token = userdata.get('WRITE_TOKEN')
finegrained_token = userdata.get('FINEGRAINED_TOKEN')

In [ ]:
import huggingface_hub
huggingface_hub.login(token=hf_token)

In [ ]:
!pip install huggingface-cli
!huggingface-cli login

ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from

In [ ]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install datasets evaluate
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -U bitsandbytes
from transformers import BitsAndBytesConfig
import math
import torch
import pathlib
from pathlib import Path
from typing import Union, Dict, List, Tuple, Optional
import numpy as np

import torch
import torch.nn as nn
from torch.nn import MultiheadAttention
from torch.nn import functional as F
from torch import Tensor

from transformers import RobertaTokenizer, RobertaTokenizerFast, RobertaModel, pipeline, AutoTokenizer, AutoModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AutoModelForMaskedLM, AutoTokenizer, AutoModelForSequenceClassification
from transformers.models.roberta.modeling_roberta import RobertaSelfAttention

from datasets import load_dataset, Dataset, load_from_disk

from peft import LoraConfig, LoraModel, PeftModel, get_peft_model, TaskType

import evaluate
import tabulate
import tokenizers
import json
import random
import os

model_checkpoint = "roberta-base"

Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-c01hbk2f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-c01hbk2f
  Resolved https://github.com/huggingface/transformers to commit ca03842cdcf2823301171ab27aec4b6b1cafdbc1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10213955 sha256=2334f8630d30f22bc66e82ef0aa2ad33e96756f7c2d7daf59261228a7306f155
  Stored in directory: /tmp/pip-ephem-wheel-cache-q6u4sy9g/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attemptin

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # Define LoRA configuration for the RoBERTa model
# lora_config = LoraConfig(
#     r=8,  # Rank of the low-rank matrices
#     lora_alpha=32,  # Scaling factor
#     target_modules=["query", "key", "value"],  # Target specific linear layers within RobertaAttention
#     modules_to_save=["LayerNorm", "classifier"],
#     lora_dropout=0.1,  # Dropout for LoRA
#     task_type=TaskType.CAUSAL_LM,  # Task type (e.g., masked language modeling), task type for MLM doesn't exist for LoRA, using CAUSAL_LM is a common workaround
#     bias="none",
#     inference_mode = False
# )

In [ ]:
dataset = load_dataset("bigbio/meddialog")
# for key in dataset.keys():
#   dataset[key] = dataset[key].shuffle().select((range(10000)))

README.md:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

meddialog.py:   0%|          | 0.00/8.09k [00:00<?, ?B/s]

bigbiohub.py:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

The repository for bigbio/meddialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/meddialog.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast = True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

### Tokenize the data

In [ ]:
def tokenize_function(example):
    dialogue = " </s> ".join(example['utterances']['utterance'])

    full_text = example['description'] + " </s> " + dialogue

    return tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=512,
        return_special_tokens_mask=True
    )

In [ ]:
tokenized_datasets = dataset['train'].map(tokenize_function, batched=False, remove_columns=["description", "utterances"])

print(tokenized_datasets[0])

Map:   0%|          | 0/482 [00:00<?, ? examples/s]

{'input_ids': [0, 26636, 415, 10, 828, 12867, 8, 236, 7, 120, 10, 205, 4356, 4438, 27028, 6, 941, 11, 1109, 9, 5, 6793, 4, 2540, 12922, 4, 33, 45, 57, 11, 1511, 19, 295, 219, 1264, 19, 5, 6793, 4, 1437, 2, 14599, 10, 828, 12867, 8, 236, 7, 120, 10, 205, 4356, 4438, 27028, 6, 941, 11, 1109, 9, 5, 6793, 4, 2540, 12922, 4, 33, 45, 57, 11, 1511, 19, 295, 219, 1264, 19, 5, 6793, 4, 1437, 2, 148, 42, 23387, 14414, 4, 14599, 2400, 64, 28, 31, 10, 22246, 642, 14599, 7910, 36, 927, 1452, 34339, 956, 238, 10, 2569, 50, 23807, 50, 97, 6793, 6, 50, 31, 103, 97, 1303, 215, 25, 26331, 50, 26570, 3277, 4, 2333, 6, 10, 621, 3681, 5, 3299, 36, 16395, 78, 43, 114, 5, 12867, 14599, 16, 15304, 12465, 6, 35583, 6, 50, 630, 75, 213, 409, 1335, 4, 47268, 808, 12, 1646, 11341, 3805, 7, 33, 21768, 6, 9641, 22246, 642, 14599, 2333, 14026, 21768, 53, 34, 55, 14599, 2400, 4, 36, 246, 73, 2146, 73, 844, 43, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### Define the data collator

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

### Define the dataloader

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets,
    batch_size=2,
    shuffle=True,
    collate_fn=data_collator
)

### Import the base model, define the LoRA configuration, and wrap it around the model

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("roberta-base")

lora_config = LoraConfig(
    task_type="MASKED_LM",
    r=16,
    lora_alpha=32,
    lora_dropout=0.1
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 589,824 || all params: 125,287,257 || trainable%: 0.4708


In [ ]:
# Define compute_metrics function -- update, I don't think we'll end up using this for MASKED_LM
import torch.nn.functional as F

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = torch.tensor(predictions)
    labels = torch.tensor(labels)

    predictions = predictions.view(-1, predictions.size(-1))
    labels = labels.view(-1)

    valid_indices = labels != -100
    predictions = predictions[valid_indices]
    labels = labels[valid_indices]

    if labels.shape[0] == 0:
        raise ValueError("No valid tokens found for loss computation.")

    loss = F.cross_entropy(predictions, labels).item()

    print("Loss:", loss)
    return {"loss": loss}

### FairLoRA? Update -- moving this to a new notebook

In [ ]:
def fairness_loss(group1_logits, group2_logits):
    prob_group1 = torch.softmax(group1_logits, dim=-1).mean()
    prob_group2 = torch.softmax(group2_logits, dim=-1).mean()
    return torch.abs(prob_group1 - prob_group2)

# Fairness regularizer
lambda_fair = 0.1

for batch in train_dataloader:
    inputs = tokenizer(batch['utterances']['utterance'], return_tensors='pt', padding=True)

    group1_mask = ...  # TODO: Add in
    group2_mask = ~group1_mask

    outputs = model(**inputs)
    logits = outputs.logits

    task_loss = loss_fn(logits, inputs['labels'])

    fairness_loss_value = fairness_loss(logits[group1_mask], logits[group2_mask])

    total_loss = task_loss + lambda_fair * fairness_loss_value
    total_loss.backward()
    optimizer.step()

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# nf4_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_use_double_quant=True,
#    bnb_4bit_compute_dtype=torch.float32
# )

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
# Necessary? Keep getting warning message

# _=model.to("cuda")

Training parameters

In [ ]:
training_args = TrainingArguments(
    output_dir="./lora_finetuned_roberta_mlm",
    eval_strategy="no",
    learning_rate=1e-5,
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    weight_decay = 0.01,
)

Train the model

In [ ]:
# Split tokenized_mlm_dataset into train and eval
# train_size = 0.8
# train_dataset, eval_dataset = tokenized_datasets["train"].train_test_split(test_size=1 - train_size).values()

# train_dataset = train_dataset.select(range(3000))
# eval_dataset = eval_dataset.select(range(95))

### Initialize the Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.379000
20,1.341200
30,1.731100
40,1.762800
50,1.442900
60,1.162400
70,1.478100
80,1.799000
90,1.547200
100,1.671500


TrainOutput(global_step=1446, training_loss=1.4293268683210614, metrics={'train_runtime': 179.0256, 'train_samples_per_second': 8.077, 'train_steps_per_second': 8.077, 'total_flos': 383301924194304.0, 'train_loss': 1.4293268683210614, 'epoch': 3.0})

## Step 5: Save and Use the LoRA Model

Save only the LoRA adapters or the full model with LoRA.

In [ ]:
trainer.push_to_hub("alexandra-barker/roberta-lora-gender-bias", token = write_token)

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alexandra-barker/lora_finetuned_roberta_mlm/commit/3b828e6b93efdb0c2985c32e2f40a0aacdb28b16', commit_message='alexandra-barker/roberta-lora-gender-bias', commit_description='', oid='3b828e6b93efdb0c2985c32e2f40a0aacdb28b16', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alexandra-barker/lora_finetuned_roberta_mlm', endpoint='https://huggingface.co', repo_type='model', repo_id='alexandra-barker/lora_finetuned_roberta_mlm'), pr_revision=None, pr_num=None)

In [ ]:
trainer.save_model('./roberta_lora_fine_tuned')
tokenizer.save_pretrained('./roberta_lora_fine_tuned')

('./roberta_lora_fine_tuned/tokenizer_config.json',
 './roberta_lora_fine_tuned/special_tokens_map.json',
 './roberta_lora_fine_tuned/vocab.json',
 './roberta_lora_fine_tuned/merges.txt',
 './roberta_lora_fine_tuned/added_tokens.json',
 './roberta_lora_fine_tuned/tokenizer.json')